<div style="background-color:	#103060; text-align:center; vertical-align: middle; padding:40px 0; margin-top:30px">
<h1 style="color:white">Website Automatisering met WP-Admin </h1>
<b style="color:white">Rogier Gernaat</b>
    </div>

<a name='start'></a>

<div style="background-color:#006069; color:white; padding: 10px;">
    <strong>Inladen van modules en library's</strong>
</div>


In [ ]:
# !pip install python-wordpress-xmlrpc
# !pip install mysql-connector-python
# !pip install pandas
# !pip install seaborn
# !pip install matplotlib
# !pip install numpy
# !pip install python-dotenv
# !pip install requests





  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)


In [20]:
import os
import mysql.connector
import pandas as pd
from dotenv import load_dotenv
from wordpress_xmlrpc import Client
from wordpress_xmlrpc.methods.posts import EditPost

<div style="background-color:#006069; color:white; padding: 10px;">
    <strong>Inladen de .env bestanden</strong>
</div>
Dit laad de gegevens in om in te loggen in de database en wp-admin

In [21]:
# Laad Azure-configuratie uit .env
load_dotenv()
db_host = os.getenv('DB_HOST')
db_port = int(os.getenv('DB_PORT', 3306))
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_ssl_ca = os.getenv('DB_SSL_CA')

# WordPress-credentials
wp_url = "https://samenzoetermeergezond.nl/xmlrpc.php"
wp_user = "SamenZoeterMeerGezond"
wp_password = "$Lmzk#S1U6gvM5p8&@@Gn2NE"

<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>SQL Verbinding Class </strong></h2>
</div>


In [22]:
class AzureDBConnector:
    def __init__(self, host, port, user, password, database, ssl_ca):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.ssl_ca = ssl_ca
        self.connection = None

    def connect(self):
        try:
            self.connection = mysql.connector.connect(
                host=self.host,
                port=self.port,
                user=self.user,
                password=self.password,
                database=self.database,
                ssl_ca=self.ssl_ca
            )
            print("Verbonden met de Azure Database.")
        except mysql.connector.Error as err:
            print(f"Fout bij het verbinden met de database: {err}")

    def fetch_query_data(self, query):
        if not self.connection:
            print("Geen actieve databaseverbinding.")
            return pd.DataFrame()
        try:
            cursor = self.connection.cursor(dictionary=True)
            cursor.execute(query)
            rows = cursor.fetchall()
            df = pd.DataFrame(rows)
            return df
        except Exception as err:
            print(f"Fout bij het uitvoeren van de query: {err}")
            return pd.DataFrame()
        finally:
            cursor.close()

    def close(self):
        if self.connection:
            self.connection.close()
            print("Verbinding met de database gesloten.")

<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>Layout maken van de pagina's </strong></h2>
</div>


In [41]:
class ActivityPublisher:
    def __init__(self, wp_url, wp_user, wp_password):
        self.client = Client(wp_url, wp_user, wp_password)

    def build_activities_html(self, activities):
        """
        Bouw de HTML voor de activiteitenkaarten. 
        Deze blijven links staan in meerdere kolommen (via flex-wrap).
        """
        html = """
        <div class="activities-container">
        """
        for _, row in activities.iterrows():
            full_desc = row.get('Extra_beschrijving', '') or ''
            shortened_desc = (full_desc[:150] + '...') if len(full_desc) > 150 else full_desc

            title = row.get('Titel', 'Onbekende titel')
            location = row.get('Locatie', 'Onbekende locatie')
            link = row.get('Link', '#')
            image_url = row.get('URL_afbeelding', '')

            alt_text = f"Afbeelding van {title}"

            card_html = f"""
            <div class="activity-card">
                <img src="{image_url}" alt="{alt_text}" loading="lazy">
                <div class="activity-content">
                    <h3 class="activity-title">{title}</h3>
                    <p class="activity-desc">{shortened_desc}</p>
                    <p class="activity-location"><strong>Locatie:</strong> {location}</p>
                    <a href="{link}" class="activity-link" target="_blank" rel="noopener">Ga hier verder</a>
                </div>
            </div>
            """
            html += card_html
        html += "</div>"
        return html

    def build_sidebar_html(self, sidebar_links):
        """
        Bouwt de sidebar met klikbare blokken, maar logo boven de tekst (vertical stacking).
        """
        html = """
        <div class="sidebar-container">
            <h3 class="sidebar-title">Andere pagina's</h3>
            <div class="sidebar-list">
        """
        for item in sidebar_links:
            name = item["name"]
            url = item["url"]
            img_url = item["img"]
            html += f"""
            <div class="sidebar-item">
                <a href="{url}" target="_blank" rel="noopener" class="sidebar-link">
                    <div class="sidebar-logo">
                        <img src="{img_url}" alt="{name}" />
                    </div>
                    <div class="sidebar-name">{name}</div>
                </a>
            </div>
            """
        html += """
            </div>
        </div>
        """
        return html

    def combine_content_with_sidebar(self, activities_html, sidebar_html):
        """
        Volledige layout: 
        - Links (flex:3) de activiteiten
        - Rechts (vast 300px) de sidebar 
        - Body met blauwe achtergrond
        """
        background_url = "https://samenzoetermeergezond.nl/wp-content/uploads/2024/07/Achtergrond-blauw.jpg"

        full_html = f"""
        <html>
        <head>
            <!-- We verwijderen de Google Fonts verwijzing, 
                 want we gebruiken nu 'astounder-squared-lc-bb' -->
            <style>
                body {{
                    margin: 0;
                    padding: 0;
                    background: url("{background_url}") no-repeat center center fixed;
                    background-size: cover;
                    /* Jouw gewenste font */
                    font-family: "astounder-squared-lc-bb", Sans-serif;
                    color: #fff;
                }}
                .page-wrapper {{
                    display: flex;
                    justify-content: space-between;
                    align-items: flex-start;
                }}
                .main-content {{
                    flex: 3;
                    padding: 20px;
                }}
                .right-sidebar {{
                    width: 300px;
                    background-color: rgba(255, 255, 255, 0.8);
                    min-height: 100vh;
                    padding: 20px;
                    box-sizing: border-box;
                    color: #000;
                }}
                /* Activiteiten: flex-wrap, meervoudige kolommen */
                .activities-container {{
                    display: flex;
                    flex-wrap: wrap;
                    gap: 16px;
                    justify-content: center;
                }}
                .activity-card {{
                    width: 280px;
                    background-color: #ffffff;
                    color: #000000;
                    border: 2px solid #ffffff; /* Outline nu wit */
                    border-radius: 8px;
                    box-shadow: 2px 2px 8px rgba(0,0,0,0.1);
                    overflow: hidden;
                    text-align: center;
                    /* Inherit of direct het nieuwe font */
                    font-family: inherit; 
                }}
                .activity-card img {{
                    width: 100%;
                    height: auto;
                    object-fit: cover;
                }}
                .activity-content {{
                    padding: 16px;
                }}
                .activity-title {{
                    font-size: 1.2em;
                    margin-bottom: 8px;
                    color: #000;
                }}
                .activity-desc {{
                    font-size: 0.9em;
                    color: #555;
                    margin-bottom: 8px;
                }}
                .activity-location {{
                    font-size: 0.8em;
                    color: #777;
                    margin-bottom: 16px;
                }}
                /* Knop iets minder dik: minder padding, iets kleinere font-size */
                .activity-link {{
                    display: inline-block;
                    padding: 8px 12px;
                    background-color: #007BFF !important;
                    color: #ffffff !important;
                    text-decoration: none;
                    border-radius: 4px;
                    font-size: 0.85em;
                    /* Hier ook jouw gewenste font, of inherit */
                    font-family: inherit; 
                    transition: background-color 0.2s ease-in-out, box-shadow 0.2s ease-in-out;
                }}
                /* Glow-effect bij hover */
                .activity-link:hover {{
                    background-color: #0056b3 !important;
                    color: #ffffff !important;
                    box-shadow: 0 0 6px rgba(0, 123, 255, 0.7);
                }}

                /* Sidebar-styling (klikbaar), logo boven tekst */
                .sidebar-container {{
                    width: 100%;
                }}
                .sidebar-title {{
                    font-size: 1.2em;
                    margin-bottom: 16px;
                    font-weight: bold;
                }}
                .sidebar-list {{
                    display: flex;
                    flex-direction: column;
                    gap: 10px;
                }}
                .sidebar-item {{
                    background: #fff;
                    color: #000;
                    border: 2px solid #cce;
                    border-radius: 4px;
                    padding: 4px;
                    width: 100%;
                    box-sizing: border-box;
                    transition: box-shadow 0.2s ease-in-out;
                    text-align: center; 
                }}
                .sidebar-item:hover {{
                    background-color: #f0f0f0;
                    box-shadow: 0 0 6px rgba(0, 123, 255, 0.7);
                }}
                .sidebar-link {{
                    display: flex;
                    flex-direction: column;
                    align-items: center;
                    text-decoration: none; 
                    color: inherit;
                }}
                .sidebar-logo {{
                    width: 100%;
                    height: 60px;
                    overflow: hidden;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    margin-bottom: 8px;
                }}
                .sidebar-logo img {{
                    width: 80%;
                    height: 100%;
                    object-fit: contain;
                }}
                .sidebar-name {{
                    font-weight: 600;
                    font-size: 0.9em;
                }}
            </style>
        </head>
        <body>
            <div class="page-wrapper">
                <div class="main-content">
                    {activities_html}
                </div>
                <div class="right-sidebar">
                    {sidebar_html}
                </div>
            </div>
        </body>
        </html>
        """
        return full_html

    def update_page_with_sidebar(self, page_id, activities, sidebar_links):
        """
        Bouwt de HTML voor de linkerkant (activiteiten) en de rechterzijbalk (klikbare blokken),
        en uploadt het naar de WordPress-pagina via XML-RPC.
        """
        try:
            activities_html = self.build_activities_html(activities)
            sidebar_html = self.build_sidebar_html(sidebar_links)
            final_html = self.combine_content_with_sidebar(activities_html, sidebar_html)

            page = {"post_content": final_html}
            self.client.call(EditPost(page_id, page))

            print(f"Pagina met ID {page_id} succesvol geüpdatet.")
        except Exception as err:
            print(f"Fout bij het updaten van de pagina: {err}")

<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>Inzet in de wijk </strong></h2>
</div>
Dit is code voor het invullen van de inzet in de wijk activiteiten op de correcte pagina

In [42]:
if __name__ == "__main__":
    # 1. Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # 2. Haal activiteiten op
    query = "SELECT Titel, Link, Beschrijving, Extra_beschrijving, Locatie, URL_afbeelding FROM activiteiten"
    activiteiten = db_connector.fetch_query_data(query)

    # 3. Sidebar-links (klikbaar)
    sidebar_links = [
        {
            "name": "ZoeterMeerActief",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ZMA-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/ZMA-e1737116617322.png"
        },
        {
            "name": "Evie.nl",
            "url": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Evie.nl_-e1737116746677.png"
        },
        {
            "name": "GGD appstore",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/GGDAPPSTORE.jpg"
        },
        {
            "name": "Nederland zorgt voor elkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-nlzve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/NLZVE-e1737116948577.png"
        },
        {
            "name": "Scheidingspunt",
            "url": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Scheidingspunt-e1737116711397.png"
        },
        {
            "name": "UITgenda",
            "url": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/UITgenda-e1737116828219.png"
        },
        {
            "name": "ZoetermeerTegenEenzaamheid",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeertegeneenzaamheid-e1737116787436.png"
        },
        {
            "name": "ZoetermeerVoorElkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeervoorelkaar-e1737116649533.png"
        }
    ]

    # 4. Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # 5. Update de pagina (deze functie krijgt nu ook sidebar_links mee)
    private_page_id = 8074  # Pas aan met de juiste pagina-ID
    publisher.update_page_with_sidebar(private_page_id, activiteiten, sidebar_links)

    # 6. Sluit de databaseverbinding
    db_connector.close()


Verbonden met de Azure Database.
Pagina met ID 8074 succesvol geüpdatet.
Verbinding met de database gesloten.


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>ZoeterMeerActief </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina

In [39]:
if __name__ == "__main__":
    # 1. Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # 2. Haal activiteiten op
    query = "SELECT Titel, Link, Extra_beschrijving, Startdatum, URL_afbeelding, Url_header_afbeelding FROM activiteiten_zma"
    activiteiten = db_connector.fetch_query_data(query)

    # 3. Sidebar-links (klikbaar)
    sidebar_links = [
        {
            "name": "ZoeterMeerActief",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ZMA-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/ZMA-e1737116617322.png"
        },
        {
            "name": "Evie.nl",
            "url": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Evie.nl_-e1737116746677.png"
        },
        {
            "name": "GGD appstore",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/GGDAPPSTORE.jpg"
        },
        {
            "name": "Nederland zorgt voor elkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-nlzve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/NLZVE-e1737116948577.png"
        },
        {
            "name": "Scheidingspunt",
            "url": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Scheidingspunt-e1737116711397.png"
        },
        {
            "name": "UITgenda",
            "url": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/UITgenda-e1737116828219.png"
        },
        {
            "name": "ZoetermeerTegenEenzaamheid",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeertegeneenzaamheid-e1737116787436.png"
        },
        {
            "name": "ZoetermeerVoorElkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeervoorelkaar-e1737116649533.png"
        }
    ]

    # 4. Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # 5. Update de pagina (deze functie krijgt nu ook sidebar_links mee)
    private_page_id = 8142  # Pas aan met de juiste pagina-ID
    publisher.update_page_with_sidebar(private_page_id, activiteiten, sidebar_links)

    # 6. Sluit de databaseverbinding
    db_connector.close()


Verbonden met de Azure Database.
Pagina met ID 8142 succesvol geüpdatet.
Verbinding met de database gesloten.
